# Importing Libraries

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import tensorflow as tf
import tensorflow_datasets as tfds
import keras

# Importing the dataset Horses or Humans

In [3]:
(train_examples, validation_examples), info = tfds.load(
    'horses_or_humans',
    split = ('train[:70%]', 'train[70%:]'),
    with_info = True,
    as_supervised = True
)

In [4]:
# Information about the dataset Horses or Humans
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes
class_names = info.features['label'].names
num_examples,num_classes, class_names

(1027, 2, ['horses', 'humans'])

# Converting the images to the required size for TensorFlow Model

In [8]:
def format_image(image, label):
  image = tf.image.resize(image, IMG_SIZE)
  return image,label

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_batches = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [9]:
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

# Creating the Model using Transfer Learning with VGG16 Model and Personal Classification Layers

In [13]:
from keras.api.applications import VGG16
from keras import layers

# Create the base model from the pre-trained model VGG16 for transfer learning
vgg16_model = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16_model.trainable = False

# Add personal classification layers on top of the base model
max_pool_layer = layers.GlobalMaxPooling2D()
prediction_layer = layers.Dense(2, activation='softmax')  # For binary classification (2 classes)

inputs = keras.Input(shape=(224, 224, 3))
x = vgg16_model(inputs)  # Use default training=True behavior
x = max_pool_layer(x)
x = layers.Dropout(0.3)(x)  # Apply dropout during training
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs, name='vgg16_model_horses_or_humans')

model.summary()

Model: "vgg16_model_horses_or_humans"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_1          │ (None, 512)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,714 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [14]:
# Create a function to implement a ModelCheckpoint callback with a specific filename
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name),
                                            monitor="val_loss",
                                            verbose=0,
                                            save_best_only=True)

In [15]:
model.compile(
    optimizer = 'adam',
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])


In [91]:
EPOCHS = 3
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data = validation_batches)

Epoch 1/3
 2/23 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.4297 - loss: 68.9393 

KeyboardInterrupt: 

In [17]:
keras.plot_model(model, to_file="horses_or_humans_vgg16.jpeg",show_shapes=True, show_dtype=True)

AttributeError: module 'keras.api' has no attribute 'plot_model'

## Save the model and history

In [18]:
model_1 = tf.keras.models.load_model('model_1.h5')
history_1 = history.history

NameError: name 'history' is not defined

In [ ]:
model_1.evaluate(validation_batches)

In [87]:
model_1.summary()

Model: "vgg16_model_horses_or_humans"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 512)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,716 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)